In [1]:
import os
import SimPEG.PF as PF
from SimPEG import (
    Maps, Regularization, Optimization, DataMisfit,
    InvProblem, Directives, Inversion, Utils
)
from SimPEG.Utils.io_utils import download
import matplotlib.pyplot as plt
import numpy as np

In [3]:
input_file = 'parameters-copy.inp'
# %% User input
# Plotting parameters, max and min densities in g/cc
vmin = -0.6
vmax = 0.6

# weight exponent for default weighting
wgtexp = 3.
# %%
# Read in the input file which included all parameters at once
# (mesh, topo, model, survey, inv param, etc.)
driver = PF.GravityDriver.GravityDriver_Inv(input_file)
# %%
# Now we need to create the survey and model information.

# Access the mesh and survey information
mesh = driver.mesh
survey = driver.survey

# define gravity survey locations
rxLoc = survey.srcField.rxList[0].locs

# define gravity data and errors
d = survey.dobs
wd = survey.std

# Get the active cells
active = driver.activeCells
nC = len(active)  # Number of active cells

# Create active map to go from reduce set to full
activeMap = Maps.InjectActiveCells(mesh, active, -100)

# Create static map
static = driver.staticCells
dynamic = driver.dynamicCells

staticCells = Maps.InjectActiveCells(
    None, dynamic, driver.m0[static], nC=nC
)
mstart = driver.m0[dynamic]

# Get index of the center
midx = int(mesh.nCx/2)
# %%
# Now that we have a model and a survey we can build the linear system ...
# Create the forward model operator
prob = PF.Gravity.GravityIntegral(mesh, rhoMap=staticCells,
                                  actInd=active)
prob.solverOpts['accuracyTol'] = 1e-4

# Pair the survey and problem
survey.pair(prob)

# Apply depth weighting
wr = PF.Magnetics.get_dist_wgt(mesh, rxLoc, active, wgtexp,
                               np.min(mesh.hx)/4.)
wr = wr**2.

Begin calculation of distance weighting for R= 3.0
Done 0.0 %
Done 10.0 %
Done 20.0 %
Done 30.0 %
Done 40.0 %
Done 50.0 %
Done 60.0 %
Done 70.0 %
Done 80.0 %
Done 90.0 %
Done 100% ...distance weighting completed!!



In [4]:
# %% Create inversion objects
reg = Regularization.Sparse(mesh, indActive=active,
                            mapping=staticCells, gradientType='total')
reg.mref = driver.mref[dynamic]
reg.cell_weights = wr * mesh.vol[active]
reg.norms = np.c_[0., 1., 1., 1.]
# reg.norms = driver.lpnorms

# Specify how the optimization will proceed
opt = Optimization.ProjectedGNCG(maxIter=20, lower=driver.bounds[0],
                                 upper=driver.bounds[1], maxIterLS=10,
                                 maxIterCG=20, tolCG=1e-3)

# Define misfit function (obs-calc)
dmis = DataMisfit.l2_DataMisfit(survey)
dmis.W = 1./wd

# create the default L2 inverse problem from the above objects
invProb = InvProblem.BaseInvProblem(dmis, reg, opt)

# Specify how the initial beta is found
betaest = Directives.BetaEstimate_ByEig(beta0_ratio=1e-2)

# IRLS sets up the Lp inversion problem
# Set the eps parameter parameter in Line 11 of the
# input file based on the distribution of model (DEFAULT = 95th %ile)
IRLS = Directives.Update_IRLS(f_min_change=1e-4, maxIRLSiter=40, beta_tol=5e-1)

# Preconditioning refreshing for each IRLS iteration
update_Jacobi = Directives.UpdatePreconditioner()

# Create combined the L2 and Lp problem
inv = Inversion.BaseInversion(invProb,
                              directiveList=[IRLS, update_Jacobi, betaest])

# %%
# Run L2 and Lp inversion
mrec = inv.run(mstart)

SimPEG.DataMisfit.l2_DataMisfit assigning default eps of 1e-5 * ||dobs||
Begin linear forward calculation: z
Done 0.0 %
Done 10.0 %
Done 20.0 %
Done 30.0 %
Done 40.0 %
Done 50.0 %
Done 60.0 %
Done 70.0 %
Done 80.0 %
Done 90.0 %
Linear forward calculation ended in: 505.9620678424835 sec

    SimPEG.InvProblem is setting bfgsH0 to the inverse of the eval2Deriv.
    ***Done using same Solver and solverOpts as the problem***
Approximated diag(JtJ) with linear operator
model has any nan: 0
=============================== Projected GNCG ===============================
  #     beta     phi_d     phi_m       f      |proj(x-g)-x|  LS    Comment   
-----------------------------------------------------------------------------
x0 has any nan: 0
   0  8.46e-03  1.64e+07  3.10e+01  1.64e+07    4.74e+02      0              
   1  4.23e-03  1.30e+05  7.16e+06  1.61e+05    3.35e+02      0              
   2  2.11e-03  8.12e+04  1.14e+07  1.05e+05    3.16e+02      0   Skip BFGS  
   3  1.06e-03  6.33e+0

In [7]:
# reconstructing l2 model mesh with air cells and active dynamic cells
L2out = activeMap * invProb.model

# reconstructing lp model mesh with air cells and active dynamic cells
Lpout = activeMap*mrec

# %%
# Plot out sections and histograms of the smooth l2 model.
# The ind= parameter is the slice of the model from top down.
yslice = midx + 1
L2out[L2out == -100] = np.nan  # set "air" to nan

In [ ]:
dataset = mesh.toVTK({'Lpout':Lpout, 'L2out':L2out})
slices = dataset.threshold(-10.).slice_orthogonal()
slices.plot(scalars='Lpout', notebook=True)

In [19]:
mesh.writeModelUBC('lpout.mod', Lpout)